# T1562.003 - Impair Command History Logging
Adversaries may impair command history logging to hide commands they run on a compromised system. Various command interpreters keep track of the commands users type in their terminal so that users can retrace what they've done. 

On Linux and macOS, command history is tracked in a file pointed to by the environment variable <code>HISTFILE</code>. When a user logs off a system, this information is flushed to a file in the user's home directory called <code>~/.bash_history</code>. The <code>HISTCONTROL</code> environment variable keeps track of what should be saved by the <code>history</code> command and eventually into the <code>~/.bash_history</code> file when a user logs out. <code>HISTCONTROL</code> does not exist by default on macOS, but can be set by the user and will be respected.

Adversaries may clear the history environment variable (<code>unset HISTFILE</code>) or set the command history size to zero (<code>export HISTFILESIZE=0</code>) to prevent logging of commands. Additionally, <code>HISTCONTROL</code> can be configured to ignore commands that start with a space by simply setting it to "ignorespace". <code>HISTCONTROL</code> can also be set to ignore duplicate commands by setting it to "ignoredups". In some Linux systems, this is set by default to "ignoreboth" which covers both of the previous examples. This means that “ ls” will not be saved, but “ls” would be saved by history. Adversaries can abuse this to operate without leaving traces by simply prepending a space to all of their terminal commands. 

On Windows systems, the <code>PSReadLine</code> module tracks commands used in all PowerShell sessions and writes them to a file (<code>$env:APPDATA\Microsoft\Windows\PowerShell\PSReadLine\ConsoleHost_history.txt</code> by default). Adversaries may change where these logs are saved using <code>Set-PSReadLineOption -HistorySavePath {File Path}</code>. This will cause <code>ConsoleHost_history.txt</code> to stop receiving logs. Additionally, it is possible to turn off logging to this file using the PowerShell command <code>Set-PSReadlineOption -HistorySaveStyle SaveNothing</code>.(Citation: Microsoft PowerShell Command History)(Citation: Sophos PowerShell command audit)(Citation: Sophos PowerShell Command History Forensics)

Adversaries may also leverage a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) on network devices to disable historical command logging (e.g. <code>no logging</code>).

## Atomic Tests

### Atomic Test #1 - Disable history collectionDisables history collection in shells
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
export HISTCONTROL=ignoreboth
whoami
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 1

### Atomic Test #2 - Disable history collection (freebsd)Disables history collection in shells
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
export HISTSIZE=0
whoami
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 2

### Atomic Test #3 - Mac HISTCONTROLThe HISTCONTROL variable is set to ignore (not write to the history file) command that are a duplicate of something already in the history 
and commands that start with a space. This atomic sets this variable in the current session and also writes it to the current user's ~/.bash_profile 
so that it will apply to all future settings as well.
https://www.linuxjournal.com/content/using-bash-history-more-efficiently-histcontrol
**Supported Platforms:** macos, linuxRun it with these steps!1. export HISTCONTROL=ignoreboth
2. echo export "HISTCONTROL=ignoreboth" >> ~/.bash_profile
3. ls
4. whoami > recon.txt


### Atomic Test #4 - Clear bash historyAn attacker may clear the bash history cache and the history file as their last act before logging off to remove the record of their command line activities. 

In this test we use the $HISTFILE variable throughout to 1. confirms the $HISTFILE variable is set 2. echo "" into it 3..5 confirm the file is empty 6 clear the history cache 7. confirm the history cache is empty. This is when the attacker would logoff.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
cp $HISTFILE $HISTFILE.OLD
if ((${#HISTFILE[@]})); then echo $HISTFILE; fi
echo "" > $HISTFILE
if [ $(wc -c <$HISTFILE) -gt 1 ]; then echo "$HISTFILE is larger than 1k"; fi
ls -la $HISTFILE 
cat $HISTFILE
history -c 
if [ $(history |wc -l) -eq 1 ]; then echo "History cache cleared"; fi
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 4

#### Cleanup: 
```bash
mv -f $HISTFILE.OLD $HISTFILE 
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Setting the HISTCONTROL environment variableAn attacker may exploit the space before a command (e.g. " ls") or the duplicate command suppression feature in Bash history to prevent their commands from being recorded in the history file or to obscure the order of commands used. 

In this test we 1. sets $HISTCONTROL to ignoreboth 2. clears the history cache 3. executes ls -la with a space in-front of it 4. confirms that ls -la is not in the history cache 5. sets $HISTCONTROL to erasedups 6. clears the history cache 7..9 executes ls -la $HISTFILE 3 times 10. confirms that their is only one command in history
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
TEST=$(echo $HISTCONTROL)
if [ "$HISTCONTROL" != "ignoreboth" ]; then export HISTCONTROL="ignoreboth"; fi
history -c 
ls -la $HISTFILE # " ls -la $HISTFILE"
if [ $(history |wc -l) -eq 1 ]; then echo "ls -la is not in history cache"; fi
# -> ls -la is not in history cache
if [ "$HISTCONTROL" != "erasedups" ]; then export HISTCONTROL="erasedups"; fi
history -c 
ls -la $HISTFILE
ls -la $HISTFILE
ls -la $HISTFILE
if [ $(history |wc -l) -eq 2 ]; then echo "Their is only one entry for ls -la $HISTFILE"; fi
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 5

#### Cleanup: 
```bash
export HISTCONTROL=$(echo $TEST)
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Setting the HISTFILESIZE environment variableAn Adversary may set the bash history files size environment variable (HISTFILESIZE) to zero to prevent the logging of commands to the history file after they log out of the system.

Note: we don't wish to log out, so we are just confirming the value of HISTFILESIZE. In this test we 1. echo HISTFILESIZE 2. set it to zero 3. confirm that HISTFILESIZE is set to zero.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
TEST=$(echo $HISTFILESIZE)
echo $HISTFILESIZE
export HISTFILESIZE=0
if [ $(echo $HISTFILESIZE) -eq 0 ]; then echo "\$HISTFILESIZE is zero"; fi
# -> $HISTFILESIZE is zero
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 6

#### Cleanup: 
```bash
export HISTCONTROL=$(echo $TEST)
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Setting the HISTSIZE environment variableAn Adversary may set the sh history files size environment variable (HISTSIZE) to zero to prevent the logging of commands to the history file after they log out of the system.

Note: we don't wish to log out, so we are just confirming the value of HISTSIZE. In this test we 1. echo HISTSIZE 2. set it to zero 3. confirm that HISTSIZE is set to zero.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo $HISTSIZE
export HISTSIZE=0
if [ $(echo $HISTSIZE) -eq 0 ]; then echo "\$HISTSIZE is zero"; fi
# -> $HISTSIZE is zero
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 7

#### Cleanup: 
```sh
export HISTSIZE=100
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Setting the HISTFILE environment variableAn Adversary may clear, unset or redirect the history environment variable HISTFILE to prevent logging of commands to the history file after they log out of the system.

Note: we don't wish to log out, so we are just confirming the value of HISTFILE. In this test we 1. echo HISTFILE 2. set it to /dev/null 3. confirm that HISTFILE is set to /dev/null.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
TEST=$(echo $HISTFILE)
echo $HISTFILE
export HISTFILE="/dev/null"
if [ $(echo $HISTFILE) == "/dev/null" ]; then echo "\$HISTFILE is /dev/null"; fi
# -> $HISTFILE is /dev/null
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 8

#### Cleanup: 
```bash
export HISTFILE=$(echo $TEST)
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Setting the HISTFILE environment variable (freebsd)An Adversary may clear, unset or redirect the history environment variable HISTFILE to prevent logging of commands to the history file after they log out of the system.

Note: we don't wish to log out, so we are just confirming the value of HISTFILE. In this test we 1. echo HISTFILE 2. set it to /dev/null 3. confirm that HISTFILE is set to /dev/null.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo $HISTFILE
export HISTFILE="/dev/null"
if [ $(echo $HISTFILE) == "/dev/null" ]; then echo "\$HISTFILE is /dev/null"; fi
# -> $HISTFILE is /dev/null
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 9

#### Cleanup: 
```sh
export HISTFILE=~/.sh_history
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Setting the HISTIGNORE environment variableAn Adversary may take advantage of the HISTIGNORE environment variable either to ignore particular commands or all commands. 

In this test we 1. set HISTIGNORE to ignore ls, rm and ssh commands 2. clear this history cache 3..4 execute ls commands 5. confirm that the ls commands are not in the history cache 6. unset HISTIGNORE variable 7.. same again, but ignoring ALL commands.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
if ((${#HISTIGNORE[@]})); then echo "\$HISTIGNORE = $HISTIGNORE"; else export HISTIGNORE='ls*:rm*:ssh*'; echo "\$HISTIGNORE = $HISTIGNORE"; fi
# -> $HISTIGNORE = ls*:rm*:ssh*
history -c 
ls -la $HISTFILE
ls -la ~/.bash_logout
if [ $(history |wc -l) -eq 1 ]; then echo "ls commands are not in history"; fi
# -> ls commands are not in history
unset HISTIGNORE

if ((${#HISTIGNORE[@]})); then echo "\$HISTIGNORE = $HISTIGNORE"; else export HISTIGNORE='*'; echo "\$HISTIGNORE = $HISTIGNORE"; fi
# -> $HISTIGNORE = *
history -c 
whoami
groups
if [ $(history |wc -l) -eq 0 ]; then echo "History cache is empty"; fi
# -> History cache is empty
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 10

#### Cleanup: 
```bash
unset HISTIGNORE
```

In [ ]:
Invoke-AtomicTest T1562.003 -TestNumbers 10 -Cleanup

## Detection
Correlating a user session with a distinct lack of new commands in their <code>.bash_history</code> can be a clue to suspicious behavior. Additionally, users checking or changing their <code>HISTCONTROL</code>, <code>HISTFILE</code>, or <code>HISTFILESIZE</code> environment variables may be suspicious.

Monitor for modification of PowerShell command history settings through processes being created with <code>-HistorySaveStyle SaveNothing</code> command-line arguments and use of the PowerShell commands <code>Set-PSReadlineOption -HistorySaveStyle SaveNothing</code> and <code>Set-PSReadLineOption -HistorySavePath {File Path}</code>. Further, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands may also be used to clear or disable historical log data with built-in features native to the network device platform.  Monitor such command activity for unexpected or unauthorized use of commands being run by non-standard users from non-standard locations.